In [ ]:
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder

root_url = "http://localhost:8000/"

# Create a user

In [ ]:
# create a user
user1_data = {
    "@id": "ark:99999/test-user1",
    "name": "Test User1",
    "type": "Person",
    "email": "testuser1@example.org",
    "password": "test1",
    "organizations": [],
    "projects": [],
    "datasets": [],
    "software": [],
    "computations": [],
    "evidencegraphs": []
}

create_user1 = requests.post(root_url + "user", data=json.dumps(user1_data))
create_user1.json()

# Get token

In [ ]:
token_session_str = requests.post(root_url + "login" + "?" + "email="+user1_data['email'] + "&password="+user1_data['password'])
print(token_session_str.json())
token = token_session_str.json()['session']

# Create an organization using Bearer token

In [ ]:
organization_data = {
    "@id": "ark:99999/test-organization",
    "@type": "Organization",
    "name": "test organization",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

headers = {'Authorization': 'Bearer ' + token}
organization_create = requests.post(root_url + f"organization/", data=json.dumps(organization_data), headers=headers)
organization_create.json()

# Create a project using Bearer token

In [ ]:
# create project
project_data = {
    "@id": "ark:99999/test-project",
    "@type": "Project",
    "name": "test project",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    },
    "memberOf": {
        "@id": organization_data['@id'],
        "@type": organization_data['@type'],
        "name": organization_data['name']
    }
}

headers = {'Authorization': 'Bearer ' + token}
project_create = requests.post(root_url + f"project/", data=json.dumps(project_data), headers=headers)
project_create.json()

# Transfer software/script to Object store with metadata

In [ ]:
from pathlib import Path, PurePath

path_to_software = '/home/sadnan/compute-test/input-script'
path = Path(path_to_software)

for p in path.rglob("*"):
    if p.is_file():
        # print(p.parent, ' - ', PurePath(p.parent).name, ' - ', p.name)
        full_path = p.resolve()

        software_data = {
            "@id": "ark:99999/test-software",
            "@type": "evi:Software",
            "name": "test software",
            "owner": {
                "@id": user1_data['@id'],
                "name": user1_data['name'],
                "email": user1_data['email'],
                "@type": "Person"
            }
        }

        software_create = requests.post(root_url + f"software/", data=json.dumps(software_data))
        software_create.json()

        software_metadata = {
            "@id": "ark:99999/test-software-"+ p.name,
            "@type": "DataDownload",
            "name": p.name,
            "encodingFormat": Path(p.name).suffix,
            "encodesCreativeWork": {
                "@id": software_data['@id'],
                "@type": software_data['@type'],
                "name": software_data['name']
            }
        }

        mp_encoder = MultipartEncoder(
            fields={
                'download': json.dumps(software_metadata),
                # plain file object, no filename or mime type produces a
                # Content-Disposition header with just the part name
                'file': ('test-software', open(full_path, 'rb'), 'text/plain'),
            }
        )
        software_upload = requests.post(root_url + f"register",
                                        data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
                                        # The MultipartEncoder provides the content-type header with the boundary:
                                        headers={'Content-Type': mp_encoder.content_type}
                                        )

        software_upload.json()

# Transfer datasets to Object store with metadata

In [ ]:
from pathlib import Path, PurePath
import uuid

datasets = []
path_to_data = "/home/sadnan/compute-test/data/Non-PreVent-hctsa"

path = Path(path_to_data)

for p in path.rglob("*"):
    if p.is_file():
        #print(p.parent, ' - ', PurePath(p.parent).name, ' - ', p.name)
        print('Transferring ', p.name, ' to ', PurePath(p.parent).name)
        unique_id = str(uuid.uuid4())
        full_path = p.resolve()
        dataset_data = {
            "@id": "ark:99999/test-data-" + f'{unique_id}',
            "@type": "Dataset",
            "name": PurePath(p.parent).name,
            "owner": {
                "@id": user1_data['@id'],
                "name": user1_data['name'],
                "email": user1_data['email'],
                "@type": "Person"
            }
        }
        # all dataset ids to be referenced in computation
        datasets.append(dataset_data['@id'])

        dataset_create = requests.post(root_url + f"dataset/", data=json.dumps(dataset_data))
        dataset_create.json()

        dataset_metadata = {
            "@id": "ark:99999/test-data-" + f'{unique_id}' + f'-' + p.name,
            "@type": "DataDownload",
            "name": p.name,
            "encodingFormat": Path(p.name).suffix,
            "encodesCreativeWork": {
                "@id": dataset_data['@id'],
                "@type": dataset_data['@type'],
                "name": dataset_data['name']
            }
        }

        mp_encoder = MultipartEncoder(
            fields={
                'download': json.dumps(dataset_metadata),
                # plain file object, no filename or mime type produces a
                # Content-Disposition header with just the part name
                'file': ('test-data', open(full_path, 'rb'), 'text/plain'),
            }
        )

        data_download_upload = requests.post(root_url + f"register",
                                             data=mp_encoder,
                                             # The MultipartEncoder is posted as data, don't use files=...!
                                             # The MultipartEncoder provides the content-type header with the boundary:
                                             headers={'Content-Type': mp_encoder.content_type}
                                             )

        data_download_upload.json()

print(datasets)

Transferring  UVA_1050_HR.csv  to  UVA_1050
Transferring  UVA_1117_HR.csv  to  UVA_1117
Transferring  UVA_1117_HR3.csv  to  UVA_1117
Transferring  UVA_1251_HR3.csv  to  UVA_1251
Transferring  UVA_1251_HR.csv  to  UVA_1251
Transferring  UVA_1119_HR.csv  to  UVA_1119
Transferring  UVA_1119_HR3.csv  to  UVA_1119
Transferring  UVA_1400_HR.csv  to  UVA_1400
Transferring  UVA_1400_HR3.csv  to  UVA_1400
Transferring  UVA_1120_HR.csv  to  UVA_1120
Transferring  UVA_1120_HR3.csv  to  UVA_1120
['ark:99999/test-data-75d4372b-8f9e-4cfa-afe6-24e3c90e8a52', 'ark:99999/test-data-08f2f4e3-76cc-4a58-bbb5-54b354cab23b', 'ark:99999/test-data-cc94381a-c2e1-4c44-855f-7ccc1811dbc8', 'ark:99999/test-data-a0077d1e-de3c-4c2a-8707-b53413731dfd', 'ark:99999/test-data-a22e91d2-8a31-46b2-beda-e82f31b99633', 'ark:99999/test-data-4b4ca9fe-620a-4b4c-a4a8-6ee437d2c373', 'ark:99999/test-data-73e28e3c-4e20-44a4-b5a7-ce19dfcb55a8', 'ark:99999/test-data-05b02b11-3a18-4554-997f-fcc85180efa3', 'ark:99999/test-data-a35dd2f7-

# Create and run computation

In [ ]:
computation_data  =  {
        "@id": "ark:99999/test-computation",
        "name":"test computation",
        "@type": "evi:Computation",
        "owner": {
                "@id": user1_data['@id'],
                "name": user1_data['name'],
                "email": user1_data['email'],
                "@type": "Person"
        },
        "container": "amancevice/pandas:latest",
        "command": "python3 /mnt/input/software/hctsa_HR_script.py",
        "usedSoftware":software_data['@id'] ,
        #"usedSoftware": {
        #        "@id": software_data['@id'],
        #        "name": software_data['name'],
        #        "@type": "evi:Software"
        #},
        "usedDataset": dataset_data['@id'] #datasets
}

computation_create = requests.post(root_url + f"computation/", data=json.dumps(computation_data))
computation_create.json()

{'created': {'@id': 'ark:99999/test-computation', '@type': 'evi:Computation'}}

In [189]:
computation_execute = requests.put(root_url + f"computation/{computation_data['@id']}/execute")
computation_execute.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [176]:
# list software
list_software = requests.get(root_url + "software")
list_software.json()

{'software': [{'@id': 'ark:99999/test-software',
   '@type': 'evi:Software',
   'name': 'test software'}]}

In [177]:
# read specific software
get_software = requests.get(root_url + f"software/{software_data['@id']}")
get_software.json()

{'@id': 'ark:99999/test-software',
 '@type': 'evi:Software',
 'distribution': [{'@id': 'ark:99999/test-software-hctsa_HR_script.py',
   '@type': 'DataDownload',
   'name': 'hctsa_HR_script.py',
   'contentUrl': 'test software/hctsa_HR_script.py'}],
 'evi:usedBy': [],
 'context': {'@vocab': 'https://schema.org/', 'evi': 'https://w3id.org/EVI#'},
 'name': 'test software',
 'owner': {'@id': 'ark:99999/test-user1',
  '@type': 'Person',
  'name': 'Test User1',
  'email': 'testuser1@example.org'}}